In [ ]:
model_checkpoint = "google/mt5-base"
batch_size = 4
train_batch_size = 16
gradient_accumulation_steps = train_batch_size // batch_size
print("gradient_accumulation_steps:", gradient_accumulation_steps)


In [ ]:
#datasets = load_dataset("squad_v2" if squad_v2 else "squad")
import pandas as pd
from tqdm.auto import tqdm
import plotly.express as px
from plotly.subplots import make_subplots
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
import collections
import os
import torch
import typing as tp
import nest_asyncio
import numpy as np
from datasets import DatasetDict, load_dataset, Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
import evaluate

squad_v2 = False # validation set

nest_asyncio.apply()

current_dir = os.getcwd()
if current_dir.endswith("code"):
    os.chdir("..")
else:
    print("current dir", current_dir)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
pad_on_right = tokenizer.padding_side == "right"

if tokenizer.cls_token_id:
    cls_token_id = tokenizer.cls_token_id
else:
    cls_token_id = tokenizer.eos_token_id

max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 256 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
import uuid
datasets = load_dataset("coastalcph/tydi_xor_rc")
def create_answers_field(example):
    return {
        **example,
        'answers': {
            'text': [example['answer'] if example['answer'] != 'no' else ''],
            'answer_start': [example['answer_start']]
        },
        'id': str(uuid.uuid4())
    }

datasets['train'] = datasets['train'].filter(lambda x: x['lang'] in ['ru', 'ja', 'fi'])
datasets['validation'] = datasets['validation'].filter(lambda x: x['lang'] in ['ru', 'ja', 'fi'])
datasets['train'] = datasets['train'].map(create_answers_field)
datasets['validation'] = datasets['validation'].map(create_answers_field)


In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        answerable = examples['answerable'][sample_index]

        # If no answers are given, set the cls_index as answer.
        if not answerable:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []
    tokenized_examples["lang"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])
        tokenized_examples["lang"].append(examples["lang"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names #[elm for elm in datasets["validation"].column_names if elm != "lang"], # we keep language column
)

datasets['validation_features'] = validation_features

In [ ]:
data_collator = default_data_collator
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)


In [ ]:
import os
os.environ["HF_TOKEN"] = open("code/hf_token.txt").read()


In [ ]:
from typing import Literal
from typing import Literal
import collections

def postprocess_qa_predictions(
    lang: Literal["ru", "ja", "fi", "all"],
    examples: Dataset, 
    features: Dataset, 
    raw_predictions: tuple[np.ndarray, np.ndarray], 
    squad_v2=True,
    n_best_size=20, 
    max_answer_length=30
) -> dict[
    str, 
    tuple[list[dict[str, str]], list[dict[str, str]]]
]:
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.defaultdict(dict) 
    
    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        example_lang = example["lang"]
        
        if lang != "all" and example_lang != lang:
            continue
        
        feature_indices = features_per_example[example_index]
        
        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.

            
            cls_index = features[feature_index]["input_ids"].index(cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example_lang][example["id"]] = answer

    # Format predictions
    formatted_predictions = {}
    formatted_ground_truths = {}
    for language, lang_predictions in predictions.items():
        formatted_predictions[language] = []
        formatted_ground_truths[language] = []
        
        for example_id, prediction in lang_predictions.items():
            # Find the corresponding example
            example = next(ex for ex in examples if ex['id'] == example_id)
            
            if squad_v2:
                formatted_predictions[language].append({
                    "id": example_id,
                    "prediction_text": prediction,
                    "no_answer_probability": min_null_score if best_answer["score"] < min_null_score else 0
                })
            else:
                formatted_predictions[language].append({
                    "id": example_id,
                    "prediction_text": prediction
                })
            
            formatted_ground_truths[language].append({
                "id": example_id,
                "answers": example['answers']
            })
    
    # If lang is not "all", return only the predictions and ground truths for that language
    if lang != "all":
        return {
            lang: (
                formatted_predictions.get(lang, []),
                formatted_ground_truths.get(lang, [])
            )
        }
    
    return {
        lang: (formatted_predictions[lang], formatted_ground_truths[lang])
        for lang in formatted_predictions.keys()
    }

def gather_predictions(
    model, 
    eval_dataset, 
) -> tuple[np.ndarray, np.ndarray]:
    model.eval()
    device = next(model.parameters()).device
    # Prepare data loader

    #gather the predictions
    all_start_logits = []
    all_end_logits = []
    
    
    with torch.no_grad():
        for batch in tqdm(
            eval_dataset.iter(batch_size=batch_size), 
            desc="generating predictions",
            total=len(eval_dataset) // batch_size + 1
        ):
            input_ids = torch.tensor(batch['input_ids'], device=device)
            attention_mask = torch.tensor(batch['attention_mask'], device=device)
            if input_ids.ndim == 1:
                input_ids = input_ids.unsqueeze(0) #add batch dimension
                attention_mask = attention_mask.unsqueeze(0) #add batch dimension
            
            outputs = model.forward(
                input_ids=input_ids, 
                attention_mask=attention_mask
            )
            
            all_start_logits.append(outputs.start_logits.cpu().numpy())
            all_end_logits.append(outputs.end_logits.cpu().numpy())


    all_start_logits = np.concatenate(all_start_logits)
    all_end_logits = np.concatenate(all_end_logits)
    return (all_start_logits, all_end_logits)


def evaluate_model(
    model, 
    eval_dataset, 
    squad_v2,
    lang : Literal["ru", "ja", "fi", "all"] = "all"
):
    
    if lang != "all":
        filtered_eval_dataset = eval_dataset.filter(lambda x: x["lang"] == lang)
        filtered_validation = datasets['validation'].filter(lambda x: x["lang"] == lang)
    else:
        filtered_eval_dataset = eval_dataset
        filtered_validation = datasets['validation']
    
    raw_predictions = gather_predictions(model, filtered_eval_dataset)
    
    final_predictions = postprocess_qa_predictions(
        lang,
        examples=filtered_validation, 
        features=filtered_eval_dataset, 
        raw_predictions=raw_predictions, 
        squad_v2=squad_v2
    )
    
    metric = evaluate.load("squad_v2" if squad_v2 else "squad")
    
    for key, value in final_predictions.items():
        #print("evaluating language:", key)
        #value is a tuple of predictions and references
        results = metric.compute(predictions=value[0], references=value[1])
        print(results)
        with open(f"eval_log.txt", "a") as f:
            f.write(f"Language: {key}\n")
            f.write(f"Results: {results}\n")
        
    return results



In [ ]:

from transformers import TrainerCallback
class CustomCallback(TrainerCallback):
    def __init__(self, eval_dataset, tokenizer, squad_v2, lang : Literal["ru", "ja", "fi", "all"] = "all"):
        self.eval_dataset = eval_dataset
        self.tokenizer = tokenizer
        self.squad_v2 = squad_v2
        self.lang = lang

    def on_epoch_end(self, args, state, control, model, **kwargs):
        print(f"Epoch {state.epoch} completed!")
        print("Evaluating model...")
        
        print("len eval_dataset:", len(self.eval_dataset))
        # Move model to the appropriate device
        
        if int(state.epoch) % 2 == 0:  # Only evaluate on even epochs
            print("Evaluating model...")
            
            print("len eval_dataset:", len(self.eval_dataset))
            results = evaluate_model(
                model, 
                self.eval_dataset, 
                self.squad_v2,
                self.lang
            )
            print(f"Evaluation results: {results}")
        else:
            print("Skipping evaluation this epoch.")


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-nlp-course",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
    report_to="wandb",
    run_name="nlp-week4-mt5-base-qa",
    gradient_accumulation_steps=gradient_accumulation_steps,
)

In [ ]:
import wandb
wandb.login(key='a3469eb2df23f67e4d6907ebacf50ffb4ee664f7')


In [ ]:
trainer = Trainer(
    model,
    args,
    callbacks=[CustomCallback(datasets["validation_features"], tokenizer, squad_v2=False)],
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()


In [ ]:
trainer.save_model("nlp-mt5-base-qa-finetune")

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

In [ ]:
trainer.push_to_hub()